In [1]:
import os 
from numpy import linspace
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from collections import defaultdict
import exifread

In [6]:
class Metadata:
    def __init__(self):
        self.img_path = None
        self.x, self.y = [], []
        self.meta_dict = defaultdict(lambda: defaultdict(int))

    def update_img_path(self, img_path):
        self.img_path = img_path

    def interpolate(self) -> interp1d:
    
        '''
        Interpolate discrete data points
        '''
        
        interpolated_y = interp1d(self.x, self.y, kind='quadratic')

        x_new = linspace(min(self.x), max(self.x), num=100)
        
        return x_new, interpolated_y(x_new)

    def plot(self) -> None:
        
        '''
        Parse dictionary and split key (metadata) and value (count)
        and plot key as x and value as y
        '''

        x_discrete, y_discrete = zip(*self.meta_dict['Focal Lengths'].items())
        x_discrete = list(int(x) for x in x_discrete)
        y_discrete = list(y_discrete)

        self.x, self.y = x_discrete, y_discrete
        x_new, y_smooth = self.interpolate()

        plt.plot(x_discrete, y_discrete, 'o', label='discrete')
        plt.plot(x_new, y_smooth, label='quadratic interpolation')

        plt.legend()
        plt.show()



# Individual Focal Lengths

In [41]:
class Node:
    def __init__(self, value: int) -> None:
        self.left = self.right = None
        self.value = value

class Tree:
    def __init__(self, root: Node=None) -> None:
        self.root = root

    def add(self, value: int) -> None:
        if self.root is None:
            self.root = Node(value)
        else:
            self._add(value, self.root)
    
    def _add(self, value: int, node: Node) -> None:
        if node.left is not None:
            if value < node.left.value:
                self._add(value, node.left)
        elif node.right is not None:
            if value > node.right.value:
                self._add(value, node.right)
        else:
            if value > node.value:
                node.right = Node(value)
            else:
                node.left = Node(value)
    
    def nearest(self, value: int, node: Node) -> int:
        
        '''
        Nearest neighbor using binary search O(log base 2 n)
        '''

        if node.left is not None:
            if value < node.left.value:
                self.nearest(value, node.left)
            
        elif node.right is not None:
            if value > node.right.value:
                self.nearest(value, node.right)
        
        else:
            return node.value


In [42]:
class Focal(Metadata):
    def __init__(self, round: bool) -> None:
        super().__init__()
        self.round = round
        self.current = None

        if round:
            primes = [18, 24, 35, 55, 85, 105, 135, 200, 300]
            
            self.meta_dict['Typical Focal Lengths'] = {
                focal : 0 for focal in primes
            }

            # Get central number for root, given sorted prime list
            root = primes[int(len(primes) / 2)]

            self.focal_tree = Tree(Node(root))
            
            # Build focal length tree
            for foc_len in primes:
                if foc_len == root:
                    pass
                else:
                    self.focal_tree.add(foc_len)

    def focal_length(self) -> int:
            
        '''
        Get focal length of current image to add new key to metadata dictionary
        or increment value of present key 
        '''

        with open(self.img_path, 'rb') as raw:
            tags = exifread.process_file(raw, stop_tag='FocalLength')
            
            self.current = tags['EXIF FocalLength'].values[0]
            self.meta_dict['Focal Lengths'][self.current] += 1

    def round_focal(self) -> int:
        
        '''
        Round focal length to nearest typical prime lens focal lengths
        using binary tree and increment value in metadata dictionary
        '''

        self.meta_dict['Typical Focal Lengths'][self.focal_tree.nearest(self.current, self.focal_tree.root)] += 1


def main() -> None:
    metadata = Focal(round=True)
    path = '/mnt/f/My Drive/Photography/2022'

    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if name.endswith('.NEF'):
                metadata.update_img_path(os.path.join(root, name))
                metadata.focal_length()
                metadata.round_focal()

    metadata.plot()

main()

KeyError: None